In [1]:
import pandas as pd
import re
import matplotlib as plt

df=pd.read_csv('new2.csv')
df.head()

,id,date,text,from,from_id,reply_to_message_id,edited,forwarded_from,contact_information,contact_vcard
0,9799,2020-06-10T22:58:32,Make a random model and calc the rmse score th...,Dragon Master,1217107241,9796.0,NaN,NaN,NaN,NaN
1,9804,2020-06-10T23:05:42,Hi all,Rohit Jain,1250027868,NaN,NaN,NaN,NaN,NaN
2,9805,2020-06-10T23:07:36,Ha my name is rohit jain and I joined your gro...,Rohit Jain,1250027868,NaN,NaN,NaN,NaN,NaN
3,9809,2020-06-10T23:18:52,I'm working on a dataset where text and it's s...,Mukul Sharma,1117859453,NaN,NaN,NaN,NaN,NaN
4,9811,2020-06-10T23:19:46,Can anyone in the group help me out... I'm ha...,Mukul Sharma,1117859453,NaN,NaN,NaN,NaN,NaN


In [2]:
df['text'] = df['text'].astype(str)

In [3]:
def message_clean(df):
    #for i in range(len(df)):
    message = ''
    if df.loc['text'][0] == '[':
        text = df.loc['text'].strip().strip('[').strip(']')        
        #print(i , ": ", text)
        escapes = ''.join([chr(char) for char in range(1, 32)])
        translator = str.maketrans('', '', escapes)
        t = text.translate(translator)
        results = re.findall(r"'(.*?)'|\"(.*?)\"|({.*?})", t)
        results = [item for elem in results for item in elem if len(item)] # Clean empty records
        for e in results:
            #print(e.strip())
            if(e[0]=='{'):
                d = eval(e)
                #print('type dict: ',d,'\n')
                #print(d['text'])
                message = message + " " + d['text'].strip()
            else:
                message = message + " " + e.strip()
    else:
        message = df.loc['text']
    #print(i , ':' , message)
    #df.iloc[i]['message'] = message
    return message

In [4]:
df['message'] = df.apply(message_clean,axis=1)
df.head()

,id,date,text,from,from_id,reply_to_message_id,edited,forwarded_from,contact_information,contact_vcard,message
0,9799,2020-06-10T22:58:32,Make a random model and calc the rmse score th...,Dragon Master,1217107241,9796.0,NaN,NaN,NaN,NaN,Make a random model and calc the rmse score th...
1,9804,2020-06-10T23:05:42,Hi all,Rohit Jain,1250027868,NaN,NaN,NaN,NaN,NaN,Hi all
2,9805,2020-06-10T23:07:36,Ha my name is rohit jain and I joined your gro...,Rohit Jain,1250027868,NaN,NaN,NaN,NaN,NaN,Ha my name is rohit jain and I joined your gro...
3,9809,2020-06-10T23:18:52,I'm working on a dataset where text and it's s...,Mukul Sharma,1117859453,NaN,NaN,NaN,NaN,NaN,I'm working on a dataset where text and it's s...
4,9811,2020-06-10T23:19:46,Can anyone in the group help me out... I'm ha...,Mukul Sharma,1117859453,NaN,NaN,NaN,NaN,NaN,Can anyone in the group help me out... I'm ha...


In [5]:
df.dropna(subset=['text'],inplace=True)

In [6]:
df[df['id']==25317]

,id,date,text,from,from_id,reply_to_message_id,edited,forwarded_from,contact_information,contact_vcard,message
9752,25317,2020-07-18T13:49:26,nan,sartaj,677791092,NaN,NaN,NaN,"{'first_name': 'Anupam', 'last_name': '', 'pho...",contacts/contact_1.vcard,nan


In [7]:
df.drop(df[df.text == 'nan'].index , inplace=True)
df.drop(df[df.text == 'NaN'].index , inplace=True)

In [8]:
df.columns

Index(['id', 'date', 'text', 'from', 'from_id', 'reply_to_message_id',
       'edited', 'forwarded_from', 'contact_information', 'contact_vcard',
       'message'],
      dtype='object')

In [9]:
df = df[['id', 'date', 'text', 'from', 'from_id', 'reply_to_message_id',
       'edited', 'forwarded_from','message']]

In [10]:
df['date'] = df.apply(lambda x: ' '.join(x['date'].split('T')),axis=1).astype('datetime64[ns]')

In [11]:
df['from'] = df['from'] + ' ' + '( ' + df['from_id'].astype(str) + ' )'

In [12]:
df.dtypes

id                              int64
date                   datetime64[ns]
text                           object
from                           object
from_id                         int64
reply_to_message_id           float64
edited                         object
forwarded_from                 object
message                        object
dtype: object

In [13]:
df.head()

,id,date,text,from,from_id,reply_to_message_id,edited,forwarded_from,message
0,9799,2020-06-10 22:58:32,Make a random model and calc the rmse score th...,Dragon Master ( 1217107241 ),1217107241,9796.0,NaN,NaN,Make a random model and calc the rmse score th...
1,9804,2020-06-10 23:05:42,Hi all,Rohit Jain ( 1250027868 ),1250027868,NaN,NaN,NaN,Hi all
2,9805,2020-06-10 23:07:36,Ha my name is rohit jain and I joined your gro...,Rohit Jain ( 1250027868 ),1250027868,NaN,NaN,NaN,Ha my name is rohit jain and I joined your gro...
3,9809,2020-06-10 23:18:52,I'm working on a dataset where text and it's s...,Mukul Sharma ( 1117859453 ),1117859453,NaN,NaN,NaN,I'm working on a dataset where text and it's s...
4,9811,2020-06-10 23:19:46,Can anyone in the group help me out... I'm ha...,Mukul Sharma ( 1117859453 ),1117859453,NaN,NaN,NaN,Can anyone in the group help me out... I'm ha...


In [14]:
final = df[['id','date','from','message','reply_to_message_id','edited','forwarded_from']]

In [15]:
final.head()

,id,date,from,message,reply_to_message_id,edited,forwarded_from
0,9799,2020-06-10 22:58:32,Dragon Master ( 1217107241 ),Make a random model and calc the rmse score th...,9796.0,NaN,NaN
1,9804,2020-06-10 23:05:42,Rohit Jain ( 1250027868 ),Hi all,NaN,NaN,NaN
2,9805,2020-06-10 23:07:36,Rohit Jain ( 1250027868 ),Ha my name is rohit jain and I joined your gro...,NaN,NaN,NaN
3,9809,2020-06-10 23:18:52,Mukul Sharma ( 1117859453 ),I'm working on a dataset where text and it's s...,NaN,NaN,NaN
4,9811,2020-06-10 23:19:46,Mukul Sharma ( 1117859453 ),Can anyone in the group help me out... I'm ha...,NaN,NaN,NaN


In [16]:
final.to_csv('finalchat.csv',index=False)